In [15]:
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets.folder import ImageFolder
from torchvision import models, transforms

import pandas as pd
from tqdm import tqdm

import utils.constants as c

In [16]:
class Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Classifier, self).__init__()
        self.backbone = models.resnet50(weights="IMAGENET1K_V2")
        for param in self.backbone.parameters():
            param.requires_grad = False

        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_classes)

    def forward(self, x):
        return self.backbone(x)



In [17]:
device = 'cpu'
src_dir = r'D:\0-Code\PG\2_sem\0_Dyplom\bridge-defects-clasification\datasets\images\test'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

dataset = ImageFolder(src_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32)

In [18]:
def load_model(pth_path):
    model = Classifier(8)
    state_dict = torch.load(pth_path, weights_only=False)
    model.load_state_dict(state_dict)
    model.eval()
    return model

my_model = load_model('output/resnet_torch_training_1.pth')

In [19]:
def predict(model: nn.Module, test_dataloader: DataLoader) -> pd.DataFrame:
    results = {
        'img': [],
        'pred_labels': []
    }
    with torch.inference_mode():
        for images, labels in tqdm(test_dataloader):
            images = images.to(device)
            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)

            results['img'].extend(os.path.basename(test_dataloader.dataset.samples[i][0])
                                        for i in range(len(predictions)))
            results['pred_labels'].extend(predictions.cpu().numpy())
    return pd.DataFrame(results)

In [20]:
predicted_df = predict(my_model, dataloader)
true_df = pd.read_json('datasets/labels/test.json')[['img', 'combination_id']]
true_df.rename(columns={'combination_id': 'ground_truth'}, inplace=True)

df = true_df.merge(predicted_df, on='img')
df

100%|██████████| 20/20 [00:39<00:00,  2.00s/it]


,img,ground_truth,pred_labels
0,image_0000264_crop_0000001.png,0,1
1,image_0000264_crop_0000001.png,0,0
2,image_0000264_crop_0000001.png,0,0
3,image_0000264_crop_0000001.png,0,0
4,image_0000264_crop_0000001.png,0,0
...,...,...,...
623,image_0000945_crop_0000001.png,0,0
624,image_0000945_crop_0000001.png,0,3
625,image_0000945_crop_0000001.png,0,1
626,image_0000945_crop_0000001.png,0,0


In [21]:
df['result'] = df['pred_labels'] == df['ground_truth']
results = df['result'].value_counts()
results

result
False    414
True     214
Name: count, dtype: int64

In [22]:
accuracy = results.iloc[1]/len(df['img'])
print(accuracy)

0.34076433121019106
